# 🧹 Clean & Validate the Unified Thyroid Dataset

This notebook takes the unified dataset produced by `unify_datasets.ipynb` and applies a step-by-step cleaning pipeline:

1. Load & inspect the raw unified file
2. Merge duplicate columns (different naming conventions across datasets)
3. Harmonise the `sex` / `gender` columns
4. Merge redundant hormone columns (`tsh_level → tsh`, etc.)
5. Reorder columns in a medically logical order
6. Remove duplicate rows
7. Fix invalid `age` values
8. Clip lab-value outliers
9. Fix measurement-flag consistency
10. Final validation checks & save

## 1. Imports & Paths

In [1]:
import os
import pandas as pd
import numpy as np

DATA_PATH = "../data/Unified Dataset/unified_dataset.csv"
OUT_PATH  = "../data/Unified Dataset/unified_dataset_cleaned.csv"

## 2. Load & Inspect

In [2]:
df = pd.read_csv(DATA_PATH, low_memory=False)
print("Shape:", df.shape)
df.head()

Shape: (225633, 61)


,age,class,country,diabetes,diagnosis,ethnicity,family_history,fti,fti_measured,ftimeasured,...,thyroidsurgery,thyroxine,tsh,tsh_level,tsh_measured,tshmeasured,tt4,tt4_measured,tt4measured,tumor
0,29,-,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,...,NaN,NaN,0.3,NaN,t,NaN,NaN,f,NaN,f
1,29,-,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,...,NaN,NaN,1.6,NaN,t,NaN,128.0,t,NaN,f
2,41,-,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,...,NaN,NaN,NaN,NaN,f,NaN,NaN,f,NaN,f
3,36,-,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,...,NaN,NaN,NaN,NaN,f,NaN,NaN,f,NaN,f
4,32,S,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,...,NaN,NaN,NaN,NaN,f,NaN,NaN,f,NaN,f


In [3]:
df.info()
missing = df.isna().mean().sort_values(ascending=False)
missing.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225633 entries, 0 to 225632
Data columns (total 61 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   age                      225633 non-null  int64  
 1   class                    225633 non-null  object 
 2   country                  212691 non-null  object 
 3   diabetes                 212691 non-null  object 
 4   diagnosis                212691 non-null  object 
 5   ethnicity                212691 non-null  object 
 6   family_history           212691 non-null  object 
 7   fti                      12140 non-null   float64
 8   fti_measured             9172 non-null    object 
 9   ftimeasured              3770 non-null    object 
 10  gender                   212691 non-null  object 
 11  goitre                   12942 non-null   object 
 12  hypopituitary            12942 non-null   object 
 13  i131_treatment           9172 non-null    object 
 14  i131

ftimeasured                0.983291
onantithyroidmedication    0.983291
queryhyperthyroid          0.983291
i131treatment              0.983291
tbgmeasured                0.983291
t4umeasured                0.983291
thyroxine                  0.983291
thyroidsurgery             0.983291
t3measured                 0.983291
t4                         0.983291
referral source            0.983291
queryonthyroxine           0.983291
queryhypothyroid           0.983291
tt4measured                0.983291
tshmeasured                0.983291
tbg                        0.981745
tt4                        0.961309
thyroid_surgery            0.959350
referral_source            0.959350
t3_measured                0.959350
dtype: float64

## 3. Merge Duplicate Measurement Columns

Several columns represent the same measurement but were named differently across the three source datasets (e.g. `tshmeasured` vs `tsh_measured`). We merge each pair using `combine_first` (keeps the non-null value) then drop the redundant column.

In [4]:
pairs = [
    ("tsh_measured", "tshmeasured"),
    ("t3_measured",  "t3measured"),
    ("tt4_measured", "tt4measured"),
    ("t4u_measured", "t4umeasured"),
    ("fti_measured", "ftimeasured"),
]

for keep, drop in pairs:
    if keep in df.columns and drop in df.columns:
        df[keep] = df[keep].combine_first(df[drop])
        df.drop(columns=[drop], inplace=True)
        print(f"Merged: {drop} -> {keep}")
    else:
        print(f"Skip (not found): {keep}, {drop}")

print("New shape:", df.shape)

Merged: tshmeasured -> tsh_measured
Merged: t3measured -> t3_measured
Merged: tt4measured -> tt4_measured
Merged: t4umeasured -> t4u_measured
Merged: ftimeasured -> fti_measured
New shape: (225633, 56)


## 4. Harmonise `sex` / `gender` Columns

Two columns encode sex information: `sex` (from thyroidDF / hypothyroid) and `gender` (from cancer_risk). We merge them into a single `sex` column with standardised `M` / `F` values.

In [5]:
# Merge gender into sex
df["sex"] = df["sex"].combine_first(df["gender"])

# Standardise values to M / F
df["sex"] = (
    df["sex"]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({"male": "M", "m": "M", "female": "F", "f": "F", "nan": np.nan})
)

# Drop the now-redundant gender column
df.drop(columns=["gender"], inplace=True)

print(df["sex"].value_counts(dropna=False))

sex
F      136077
M       89099
NaN       457
Name: count, dtype: int64


## 5. Merge Redundant Hormone Columns

The cancer_risk dataset stores hormone values under `tsh_level`, `t3_level`, `t4_level`, while the other datasets use `tsh`, `t3`, `t4`. We consolidate these into a single column each.

In [6]:
# TSH
df["tsh"] = df["tsh"].combine_first(df["tsh_level"])
df.drop(columns=["tsh_level"], inplace=True)

# T3
df["t3"] = df["t3"].combine_first(df["t3_level"])
df.drop(columns=["t3_level"], inplace=True)

# T4
df["t4"] = df["t4"].combine_first(df["t4_level"])
df.drop(columns=["t4_level"], inplace=True)

print("Hormone columns merged. Shape:", df.shape)

Hormone columns merged. Shape: (225633, 52)


## 6. Merge Remaining Duplicate Column Pairs

A similarity scan identified more pairs with >80% name overlap. We merge all of them now.

In [7]:
# Merge on_thyroxine ← thyroxine
df["on_thyroxine"] = df["on_thyroxine"].combine_first(df["thyroxine"])
df.drop(columns=["thyroxine"], inplace=True)

# Remaining pairs
pairs_to_merge = [
    ("referral_source",      "referral source"),
    ("i131_treatment",       "i131treatment"),
    ("query_hyperthyroid",   "queryhyperthyroid"),
    ("query_hypothyroid",    "queryhypothyroid"),
    ("query_on_thyroxine",   "queryonthyroxine"),
    ("tbg_measured",         "tbgmeasured"),
]

for keep, drop in pairs_to_merge:
    if keep in df.columns and drop in df.columns:
        df[keep] = df[keep].combine_first(df[drop])
        df.drop(columns=[drop], inplace=True)
        print(f"✅ Merged {drop} -> {keep}")
    else:
        print(f"⚠️  Skip (missing): {keep} / {drop}")

print("✅ Done. New shape:", df.shape)

✅ Merged referral source -> referral_source
✅ Merged i131treatment -> i131_treatment
✅ Merged queryhyperthyroid -> query_hyperthyroid
✅ Merged queryhypothyroid -> query_hypothyroid
✅ Merged queryonthyroxine -> query_on_thyroxine
✅ Merged tbgmeasured -> tbg_measured
✅ Done. New shape: (225633, 45)


## 7. Reorder Columns in Medical Order

Organise columns into logical clinical groups: Identifiers → Demographics → Risk Factors → Clinical Conditions → Treatment → Lab Values → Measurement Flags → Queries → Target.

In [8]:
new_order = [
    # Identifiers
    "patient_id", "source",
    # Demographics
    "age", "sex", "country", "ethnicity",
    # Risk factors
    "family_history", "iodine_deficiency", "diabetes", "obesity",
    "lithium", "psych", "pregnant",
    # Clinical conditions
    "goitre", "hypopituitary", "tumor", "nodule_size", "diagnosis",
    # Treatment
    "on_thyroxine", "on_antithyroid_meds",
    "onantithyroidmedication", "i131_treatment",
    # Lab values
    "tsh", "t3", "tt4", "t4", "t4u", "fti",
    # Measurement flags
    "tsh_measured", "t3_measured",
    "tt4_measured", "t4u_measured", "fti_measured",
    # Queries
    "query_hyperthyroid", "query_hypothyroid",
    "query_on_thyroxine",
    # Target
    "class"
]

# Only keep columns that actually exist in the dataframe
new_order = [c for c in new_order if c in df.columns]
df = df[new_order]

print("Columns organised medically ✅")
print("Final column list:", df.columns.tolist())

Columns organised medically ✅
Final column list: ['patient_id', 'source', 'age', 'sex', 'country', 'ethnicity', 'family_history', 'iodine_deficiency', 'diabetes', 'obesity', 'lithium', 'psych', 'pregnant', 'goitre', 'hypopituitary', 'tumor', 'nodule_size', 'diagnosis', 'on_thyroxine', 'on_antithyroid_meds', 'onantithyroidmedication', 'i131_treatment', 'tsh', 't3', 'tt4', 't4', 't4u', 'fti', 'tsh_measured', 't3_measured', 'tt4_measured', 't4u_measured', 'fti_measured', 'query_hyperthyroid', 'query_hypothyroid', 'query_on_thyroxine', 'class']


## 8. Remove Duplicate Rows

In [9]:
print("Duplicate rows before:", df.duplicated().sum())

df = df.drop_duplicates()

print("New shape:", df.shape)
print("Duplicate rows after:", df.duplicated().sum())

Duplicate rows before: 65
New shape: (225568, 37)
Duplicate rows after: 0


## 9. Fix Invalid Age Values

Ages above 100 are physiologically implausible and are treated as data entry errors — they are set to `NaN`.

In [10]:
print("Age distribution before fix:")
print(df["age"].describe())
print("\nRows with age > 100:", (df["age"] > 100).sum())

df.loc[df["age"] > 100, "age"] = None

print("\nAge distribution after fix:")
print(df["age"].describe())

Age distribution before fix:
count    225568.000000
mean         52.797179
std         239.708087
min           1.000000
25%          33.000000
50%          52.000000
75%          70.000000
max       65526.000000
Name: age, dtype: float64

Rows with age > 100: 5

Age distribution after fix:
count    225563.000000
mean         51.922944
std          21.488305
min           1.000000
25%          33.000000
50%          52.000000
75%          70.000000
max          97.000000
Name: age, dtype: float64


## 10. Clip Lab Value Outliers

Extreme outliers in lab measurements are clipped to the 1st–99th percentile range to reduce the impact of measurement errors or typos.

In [11]:
# Summary before clipping
df[["tsh", "t3", "t4", "tt4", "fti", "t4u"]].describe()

,tsh,t3,t4,tt4,fti,t4u
count,224726.000000,222964.000000,216396.000000,8730.000000,12075.000000,12068.000000
mean,5.045361,1.994643,9.876084,108.700305,109.730579,0.955257
std,6.209527,0.872285,13.655944,37.522670,42.830357,0.252386
min,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,2.300000,1.260000,6.390000,87.000000,92.000000,0.860000
50%,4.830000,2.000000,8.300000,104.000000,108.000000,0.960000
75%,7.430000,2.730000,10.210000,126.000000,126.000000,1.070000
max,530.000000,18.000000,430.000000,600.000000,881.000000,2.330000


In [12]:
for col in ["tsh", "t3", "t4", "tt4", "fti", "t4u"]:
    if col in df.columns:
        q01 = df[col].quantile(0.01)
        q99 = df[col].quantile(0.99)
        df[col] = df[col].clip(q01, q99)

print("Lab values clipped to [1%, 99%] ✅")

Lab values clipped to [1%, 99%] ✅


## 11. Fix Measurement-Flag Consistency

If a lab's `_measured` flag is `'f'` (false = not measured), then the corresponding numeric value should be `NaN`. This enforces consistency between flags and values.

In [13]:
measurement_pairs = [
    ("tsh_measured", "tsh"),
    ("t3_measured",  "t3"),
    ("tt4_measured", "tt4"),
    ("fti_measured", "fti"),
    ("t4u_measured", "t4u"),
]

for flag, value in measurement_pairs:
    if flag in df.columns and value in df.columns:
        df.loc[df[flag] == "f", value] = None

print("Measurement consistency fixed ✅")

Measurement consistency fixed ✅


## 12. Final Validation Checks

In [14]:
# Check 1: No lab values where flag says 'not measured'
checks = {
    "TSH": df[(df["tsh_measured"] == "f") & (df["tsh"].notna())].shape[0],
    "T3" : df[(df["t3_measured"]  == "f") & (df["t3"].notna())].shape[0],
    "TT4": df[(df["tt4_measured"] == "f") & (df["tt4"].notna())].shape[0],
    "FTI": df[(df["fti_measured"] == "f") & (df["fti"].notna())].shape[0],
    "T4U": df[(df["t4u_measured"] == "f") & (df["t4u"].notna())].shape[0],
}
print("Measurement flag inconsistencies (should all be 0):", checks)

# Check 2: No male patients marked as pregnant
male_pregnant = df[(df["sex"] == "M") & (df["pregnant"].isin(["t", True, 1]))].shape[0]
print("Male patients marked pregnant (should be 0):", male_pregnant)

# Check 3: No negative lab values
neg = {col: df[df[col] < 0].shape[0]
       for col in ["tsh", "t3", "t4", "tt4", "fti", "t4u", "nodule_size"]
       if col in df.columns}
print("Negative lab values (should all be 0):", neg)

# Check 4: Diagnosis vs class cross-tab
if "diagnosis" in df.columns:
    display(pd.crosstab(df["diagnosis"], df["class"]))

Measurement flag inconsistencies (should all be 0): {'TSH': 0, 'T3': 0, 'TT4': 0, 'FTI': 0, 'T4U': 0}
Male patients marked pregnant (should be 0): 0
Negative lab values (should all be 0): {'tsh': 0, 't3': 0, 't4': 0, 'tt4': 0, 'fti': 0, 't4u': 0, 'nodule_size': 0}


class,High,Low,Medium
diagnosis,,,
Benign,9564,92153,61479
Malignant,22339,16235,10921


## 13. Save the Final Cleaned Dataset

In [15]:
output_path = "../data/Unified Dataset/unified_dataset_validated.csv"

df.to_csv(output_path, index=False)

print("Dataset saved successfully ✅")
print("Location:", output_path)
print("Final shape:", df.shape)

Dataset saved successfully ✅
Location: ../data/Unified Dataset/unified_dataset_validated.csv
Final shape: (225568, 37)
